# Demo notebook for Model Reader

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Delete the time files before running.
model = 'Ovation-Prime'
variables_requested = ['Phi_eEall', 'Phi_iNall','Eave_e_diff']  # one variable from each coordinate
# The second variable is included here just for reader testing not validation.
# change file path to where data is stored on your machine
#file_dir = 'D:/Weimer/Dibyendu_Sur_071122_IT_1/'
file_dir = "/Users/lrastaet/Kamodo_data/OvationPrime/Lutz_Rastaetter_021524_IT_1/output/"
from os.path import isfile
from os import remove
if isfile(file_dir+model+'_times.txt'):
    remove(file_dir+model+'_times.txt')
if isfile(file_dir+model+'_list.txt'):
    remove(file_dir+model+'_list.txt')

In [ ]:
# confirm MW functions work appropriately with model varnames dictionary
import kamodo_ccmc.flythrough.model_wrapper as MW
#MW.Variable_Search('flux')

In [ ]:
#MW.Variable_Search('flux', model)

In [ ]:
MW.Variable_Search('potential', model, file_dir)

In [ ]:
MW.Var_3D(model)

In [ ]:
MW.Var_units(model, variables_requested)

In [ ]:
# Confirm time method works with model reader
MW.File_Times(model, file_dir)

In [ ]:
# Confirm file list method works with model reader
MW.File_List(model, file_dir)

In [ ]:
# Check that time files creation works, that reader works for one variable,
# and that an unknown variable request does not break it.
reader = MW.Model_Reader(model)
kamodo_object = reader(file_dir, variables_requested=['Trash'])
kamodo_object

In [ ]:
# Check that reader works for one variable with an unknown variable
kamodo_object = reader(file_dir) #, variables_requested=['Trash', variables_requested[0]])
kamodo_object

In [ ]:
# Test model reader with one variable from each coordinate system
kamodo_object = reader(file_dir, variables_requested=variables_requested)
kamodo_object

In [ ]:
# Check that reading the time files works and that reader works for one variable,
kamodo_object = reader(file_dir, variables_requested=variables_requested[:1])
kamodo_object

In [ ]:
# Confirm that interpolation works. 
from math import isnan
print(kamodo_object.Phi_eEall([0.5, 10., 60.]))
if isnan(kamodo_object.Phi_eEall([0.5, 10., 60.])[0]):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')
print(kamodo_object.Phi_eEall_ijk(time=0.5, lon=10., lat=60.))
if isnan(kamodo_object.Phi_eEall_ijk(time=0.5, lon=10., lat=60.)):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')
if not kamodo_object.Phi_eEall([0.5, 10., 60.]) == kamodo_object.Phi_eEall_ijk(time=0.5, lon=10., lat=60.):
    raise AttributeError('Values are not equal.')
else:
    print('Values are equal.')
print(kamodo_object.Phi_eEall_ijk(time=0.5).shape)

In [ ]:
# Check that reading the time files works, and that the reader works for all variables
kamodo_object = reader(file_dir)
kamodo_object

In [ ]:
# Get a list of all of the functionalized variables, both regular and gridded
var_list = list(MW.Variable_Search('', model, file_dir, return_dict=True).keys())
varijk_list = sorted(var_list + [item+'_ijk' for item in var_list])
varijk_list

In [ ]:
# Test coordinate range logic for all variables
MW.Coord_Range(kamodo_object, varijk_list)

In [ ]:
# Check that the kamodo object was built properly.
print(kamodo_object.Phi_eEall([0.5, 10., 60.]))
if isnan(kamodo_object.Phi_eEall([0.5, 10., 60.])[0]):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')

In [ ]:
# Check that the reader works for the testing subset
kamodo_object = reader(file_dir, variables_requested=variables_requested)
kamodo_object

In [ ]:
# Check that the reader works for the testing subset
kamodo_object = reader(file_dir, variables_requested='all')
kamodo_object

In [ ]:
# Confirm that the interpolator works for each testing variable and type
print(kamodo_object.Phi_eEall([0.5, 10., 60.]))
print(kamodo_object.Phi_eEall_ijk(time=0.5, lon=10., lat=60.))
if not kamodo_object.Phi_eEall([0.5, 10., 60.]) == kamodo_object.Phi_eEall_ijk(time=0.5, lon=10., lat=60.):
    raise AttributeError('Values are not equal.')
else:
    print('Values are equal.')
print(kamodo_object.Phi_eEall_ijk(time=0.5).shape)

In [ ]:
# Generate a plot for validation
from kamodo_ccmc.tools.plotfunctions import toColor
toColor(kamodo_object.plot('Phi_eEall_ijk', plot_partial={'Phi_eEall_ijk': {'lat': 60.}}),
        colorscale="Viridis")

In [ ]:
# Generate a plot for validation
fig1 = toColor(kamodo_object.plot('Phi_eEall_ijk', plot_partial={'Phi_eEall_ijk': {'time': 0.5}}),
               colorscale="Viridis")
fig1

In [ ]:
kamodo_object.var_dict

In [ ]:
# create polar plot for validation at the poles
import kamodo_ccmc.tools.plotfunctions as pf
utcts = kamodo_object.filedate.timestamp() + 0.5*3600.  # time in UTC seconds
fig1_copy = fig1
print(fig1)
pf.ReplotLL3D(fig1_copy, model, 400, utcts, plotCoord='GEO',
              opacity=0.85, colorscale='Viridis', axis=True, title=model+' Phi_eEall_ijk')
print(fig1_copy)

In [ ]:
# Test that more than one variable works through the flythrough
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
import datetime as dt
start_utcts = dt.datetime(2014, 1, 1, 0, 30).replace(tzinfo=dt.timezone.utc).timestamp()
end_utcts = dt.datetime(2014, 1, 1, 1, 30).replace(tzinfo=dt.timezone.utc).timestamp()-1
results = SF.ModelFlythrough(model, file_dir, [variables_requested[0]], [start_utcts, end_utcts], [0., 180.], [60., -60.],
                             [400., 400.], 'GDZ-sph')
results[variables_requested[0]]

In [ ]:
# Test that one variable works through the flythrough
results = SF.ModelFlythrough(model, file_dir, [variables_requested[0]], [start_utcts], [0.], [60.],
                             [400.], 'GDZ-sph')
results[variables_requested[0]]